In [20]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

In [21]:
from tensorboardX import SummaryWriter 
#SummaryWriter 是一個類別，包含這套件的所有功能。

writer = SummaryWriter(comment='test')
#在預設資料夾後面加上註解 檔名變為：'runs/Aug20-obov01-3xLR'

In [22]:
if torch.cuda.is_available():
  print("is available")

is available


In [23]:
class Model(nn.Module):
  def __init__(self, n):
    super(Model,self).__init__()
    self.linear = nn.Linear(n,1,bias=False)
    self.criterion = nn.MSELoss(reduction='mean')
  def forward(self, x):
    output = self.linear(x).squeeze(1)
    return output

  def cal_loss(self, pred, target):
    ''' Calculate loss '''
    # TODO: you may implement L1/L2 regularization here
    
    return self.criterion(pred, target)
  

In [24]:
def train(tr_set, dv_set, model, config, device):
    ''' DNN training '''

    n_epochs = config['n_epochs']  # Maximum number of epochs

    # Setup optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas'])

    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    iter_idx = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(device), y.to(device)   # move data to device (cpu/cuda)
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            loss_record['train'].append(mse_loss.detach().cpu().item())
            writer.add_scalar('myscalar', mse_loss.detach().cpu().item(), iter_idx)
            iter_idx+=1
        # After each epoch, test your model on the validation (development) set.
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            # Save model if your model improved
            min_mse = dev_mse
            print('Saving model (epoch = {:4d}, loss = {:.4f})'
                .format(epoch + 1, min_mse))
            torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1

        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return min_mse, loss_record

In [25]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
        
    
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss



In [26]:


def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()                 # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = True                    # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 150,               # mini-batch size for dataloader
    'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        # 'lr': 0.001,                 # learning rate of SGD
        # 'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth'  # your model will be saved here
}

In [27]:
model = Model(4).to(device)  # Construct model and move to device

In [28]:
class RandomDataset(Dataset):
    ''' Dataset for loading and preprocessing the COVID19 dataset '''
    def __init__(self,
                 data,
                 mode='train',
                 target_only=False):
        self.mode = mode

        target = data[1]
        data = data[0]
        
        # Splitting training data into train & dev sets
        if mode == 'train':
            indices = [i for i in range(len(data)) if i % 10 != 0]
        elif mode == 'dev':
            indices = [i for i in range(len(data)) if i % 10 == 0]
        
        # Convert data into PyTorch tensors
        self.data = data[indices].type(torch.FloatTensor)
        # self.data = torch.FloatTensor(data[indices])
        self.target = target[indices].type(torch.FloatTensor)
        # self.target = torch.FloatTensor(target[indices])
        
        self.dim = self.data.shape[1]

        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        # Returns one sample at a time
        if self.mode in ['train', 'dev']:
            # For training
            return self.data[index], self.target[index]
        else:
            # For testing (no target)
            return self.data[index]

    def __len__(self):
        # Returns the size of the dataset
        return len(self.data)

In [29]:
# dataset = RandomDataset((tr_x,tr_y), 'train', target_only=False)
# a,b=dataset[0]
# print(a,b)

In [30]:
def prep_dataloader(data, mode, batch_size, n_jobs=0, target_only=False):
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = RandomDataset(data, mode=mode, target_only=target_only)  # Construct dataset
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode == 'train'), drop_last=False,
        num_workers=n_jobs, pin_memory=True)                            # Construct dataloader
    return dataloader

In [31]:
tr_x = torch.randint(100,(3000,4))
tr_y = torch.sum(tr_x, 1)




tr_set = prep_dataloader((tr_x,tr_y), 'train', config['batch_size'], target_only=target_only)
dv_set = prep_dataloader((tr_x,tr_y), 'dev', config['batch_size'], target_only=target_only)



Finished reading the train set of COVID19 Dataset (2700 samples found, each dim = 4)
Finished reading the dev set of COVID19 Dataset (300 samples found, each dim = 4)


In [32]:
# for x in tr_set:
#   print(x[0])

In [33]:
model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)

Saving model (epoch =   78, loss = 973.2803)
Saving model (epoch =   79, loss = 913.9248)
Saving model (epoch =   80, loss = 858.1772)
Saving model (epoch =   81, loss = 806.0389)
Saving model (epoch =   82, loss = 757.4006)
Saving model (epoch =   83, loss = 712.1854)
Saving model (epoch =   84, loss = 670.0483)
Saving model (epoch =   85, loss = 630.5217)
Saving model (epoch =   86, loss = 593.6559)
Saving model (epoch =   87, loss = 559.2103)
Saving model (epoch =   88, loss = 527.4413)
Saving model (epoch =   89, loss = 497.5665)
Saving model (epoch =   90, loss = 470.1275)
Saving model (epoch =   91, loss = 444.4730)
Saving model (epoch =   92, loss = 420.7288)
Saving model (epoch =   93, loss = 398.7556)
Saving model (epoch =   94, loss = 378.2859)
Saving model (epoch =   95, loss = 359.3631)
Saving model (epoch =   96, loss = 341.8519)
Saving model (epoch =   97, loss = 325.6729)
Saving model (epoch =   98, loss = 310.6552)
Saving model (epoch =   99, loss = 296.6462)
Saving mod

In [15]:
for param in model.parameters():
  print(param.data)

tensor([[1., 1., 1., 1.]], device='cuda:0')


In [16]:
m = Model(4)
m.load_state_dict(torch.load('models/model.pth'))
for param in m.parameters():
  print(param.data)

tensor([[1., 1., 1., 1.]])
